In [ ]:
import os
os.chdir('/mmsegmentation')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from mmseg.apis import init_model, inference_model, show_result_pyplot
import mmcv
import cv2

# LOAD MODEL

config_file = '/configs/model_config.py'

# CHECKPOINT

checkpoint_file = 'work_dirs/best_checkpoint.pth'

# DEVICE

device = 'cuda:0'

model = init_model(config_file, checkpoint_file, device=device)

img_path = '/automine1d_distortion/haz_luz/img/c06dd03c-1663920795.600000.png'
ann_path = '/automine1d_distortion/haz_luz/ann/c06dd03c-1663920795.600000.png'

img_bgr = cv2.imread(img_path)
ann_bgr = cv2.imread(ann_path)

# VISUALIZE IMAGE
plt.figure(figsize=(8, 8))
plt.imshow(img_bgr[:, :, ::-1])  # Convert BGR to RGB
plt.savefig('outputs/img.jpg', bbox_inches='tight', pad_inches=0)  # Save image
plt.show()

# VISUALIZE MASK
ann_visible = (ann_bgr * 255).astype(np.uint8)

plt.figure(figsize=(8, 8))
plt.imshow(ann_visible[:, :, ::-1])  # If in BGR, reverse channels
plt.axis('off')
plt.savefig('outputs/ann.jpg', bbox_inches='tight', pad_inches=0)  # Save annotation
plt.show()

# INFERENCE
result = inference_model(model, img_bgr)

pred_mask = result.pred_sem_seg.data[0].cpu().numpy()

np.unique(pred_mask)

# DISPLAY PREDICTED MASK
plt.figure(figsize=(8, 8))
plt.imshow(pred_mask)
plt.show()

# Assuming the road corresponds to class 1
road_class = 1

# Create a binary mask: 255 where road, 0 elsewhere
binary_mask = np.where(pred_mask == road_class, 255, 0).astype(np.uint8)

# Visualize binary mask
plt.figure(figsize=(8, 8))
plt.imshow(binary_mask, cmap='gray')  # Display as grayscale image
plt.axis('off')  # Optional: hide axes
plt.savefig('outputs/pred.jpg', bbox_inches='tight', pad_inches=0)  # Save prediction
plt.show()
 